TODO
1. develop kernel 
2. validate kernel with h-param variation as in ref#2

In [57]:
import numpy as np
import pandas as pd
import logging
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr
from numpy.linalg import cholesky, det, lstsq,inv
logging.basicConfig(level=logging.INFO)

In [2]:
def kernel(xp,xq,p,q,lambda_diag,v,sigma_n):
    logger = logging.getLogger()
    """
    xp: float array mx1 vector of input point p
    xq : float array
    lambda_ 1Xm represents the diagonal of the hyperparemter diagonal matrix lambda_
    v: hyperparemter
    sigma2_n : hyperparameter
    """
    #TODO check all inputes are doubles
    lambda_power_minus_1 = np.diag(np.reciprocal(lambda_diag))
    exponent_ = np.matmul(np.matmul(-(xp-xq).T,lambda_power_minus_1),(xp-xq))
    delta_pq = 1.0 if p==q else 0.0
    kernel = v**2 * np.exp(-0.5*exponent)+delta_pq*sigma_n**2
    #logging
    logger.debug(f'lambda diag= {lambda_diag}')
    logger.debug(f'lambda_pow_minus_1 {lambda_power_minus_1}')
    logger.debug(f'term1 = -(xp-xq).T * lambda_pow_minus_1 = {term1}')
    logger.debug(f'exponent = {exponent_}')
    ######
    return kernel

In [3]:
# don't delete
xp = pd.Series([1.0,13.0]).values.reshape(-1,1)
xq = pd.Series([1.0,2.0]).values.reshape(-1,1)
lambda_diag = [0.2,0.3]
lambda_power_minus_1 = np.diag(np.reciprocal(lambda_diag))
lambda_power_minus_1
(xp-xq).T.shape
term1 = np.matmul(-(xp-xq).T,lambda_power_minus_1)
exponent = np.matmul(term1,(xp-xq))

In [4]:
k= kernel(xp=xp,xq=xq,p=1,q=2,lambda_diag=lambda_diag,sigma_n=0.05,v=1.0)
k

array([[6.83217475e-176]])

In [5]:
# quick validation :kernel vs euclidean distance , comment out , time consuming
# dim = 100,5
# X = np.random.normal(loc=0,scale=1.0,size=dim)
# k = np.zeros(dim[0]**2)
# euc = np.zeros(dim[0]**2)
# idx = 0
# for i in np.arange(dim[0]):
#     for j in np.arange(dim[0]):
#         xp = X[i,].reshape(-1,1)
#         xq = X[j,].reshape(-1,1)
#         k[idx] = kernel(xp=xp,xq=xq,p=i,q=j,lambda_diag=[0.2,0.1,0.3,0.4,0.5],sigma_n=0.05,v=1.0)
#         euc[idx] = euclidean(u=xp,v=xq)
#         idx = idx + 1
# pearsonr(x=k,y=np.reciprocal(euc+0.01))
# if corr is +ve and significant, then we are good

(0.9999074085159615, 0.0)

In [6]:
# test scipy minimize

In [7]:
from scipy.optimize import minimize
def fn(x,x0,c):
    return np.sum((x-x0)**2)+c
x0 = np.array([[1,1]])
x = np.array([[2,2]])

x_min = minimize(fun=fn,x0=np.array([[0,0]]),args=(x0,10))
x_min.x

array([1., 1.])

In [8]:
# def kernel as matrix operation

In [9]:
# Note: in http://krasserm.github.io/2018/03/19/gaussian-processes/
# in the implementatoin of Gaussian kernel , the sum of the two squared terms is broadcast sum
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.add.html
# https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html
# why ? as in first term , reshape(-1,1) is used , then the row sum shape is converted from (m,) to (m,1)
# then it is summed with array of shape (n,) then shapes are not equal, even if m==n , then the second dimensions 
# are not equal
import numpy as np
def kernel(X1, X2, l=1.0, sigma_f=1.0):
    ''' Isotropic squared exponential kernel. Computes a covariance matrix from points in X1 and X2. Args: X1: Array of m points (m x d). X2: Array of n points (n x d). Returns: Covariance matrix (m x n). '''
    # the sume is broad cast 
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

X1 = np.random.normal(loc = 10,scale = 2, size = (10,2))
X2 = np.random.normal(loc = 10,scale = 2, size = (10,2))
X1_sum= np.sum(X1**2, 1).reshape(-1,1)
X2_sum= np.sum(X2**2, 1)
X2_sum_2= np.sum(X2**2, 1).reshape(-1,1)
tot = X1_sum + X2_sum
print((X1_sum + X2_sum).shape)
print((X1_sum + X2_sum_2).shape)

(10, 10)
(10, 1)


In [29]:
def kernel_mtx(X1,X2,lambda_diag,v,sigma_n):
    ## from http://krasserm.github.io/2018/03/19/gaussian-processes/
    ## sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    
    logger = logging.getLogger()
    logger.setLevel(level=logging.INFO)
    assert X1.shape[1] == X2.shape[1]
    assert X1.shape[1] == len(lambda_diag)
    
    
    lambda_power_minus_1 = np.diag(np.reciprocal(lambda_diag))
    # quadratic terms are scale by inverse precision (variance) , precision = lambda (Captial case)
    X1_sq_sum_scaled = np.sum(a=np.matmul(X1**2,lambda_power_minus_1),axis=1)
    X2_sq_sum_scale = np.sum(a=np.matmul(X2**2,lambda_power_minus_1),axis=1)
    X1_matmul_X2_T_scaled = np.matmul(np.matmul(X1,lambda_power_minus_1),X2.T)
    
    sqdist_scaled = X1_sq_sum_scaled.reshape(-1,1)+X2_sq_sum_scale-2*X1_matmul_X2_T_scaled
    sigma_n_sq = np.zeros((X1.shape[0],X2.shape[0]))
    if X1.shape[0] == X2.shape[0]: #same data-set , or at least comparable
        sigma_n_sq = np.diag(np.repeat(sigma_n**2,X1.shape[0]))
        
    kernel = v**2 * np.exp(-0.5*sqdist_scaled)+sigma_n_sq
    logger.debug(f'sqdist_scaled = {sqdist_scaled}')
    return kernel


M = 10
N = 8
D = 3
X1 = np.random.normal(loc = 10, scale  = 2, size = (M,D))
X2 = np.random.normal(loc = 5, scale  = 1, size = (N,D))
lambda_diag = np.random.normal(loc = 1, scale = 0.01, size = D)
v = 1
sigma_n = 0.05
X1.shape
k = kernel_mtx(X1,X1,lambda_diag,v,sigma_n)
np.diag(k)

array([1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025,
       1.0025, 1.0025])

In [55]:
def kernel_fn(theta,X1,X2):
    assert X1.shape[1] == X2.shape[1]
    D = X1.shape[1]
    lambda_diag=theta[0:D]
    v= theta[D]
    sigma_n = theta[D+1]
    return kernel_mtx(X1,X1,lambda_diag,v,sigma_n)

In [72]:
def neg_ll(theta,X,t,kernel_fn):
    assert X.shape[0]==t.shape[0]
    K = kernel_fn(theta,X,X)
    #Y_train.T.dot(inv(K).dot(Y_train)
    neg_ll = np.log(det(K))+t.T.dot(inv(K).dot(t))
    return neg_ll[0][0]
t = np.random.normal(loc=5,scale = 1,size = (M,1))
neg_ll(X=X1,t=t,kernel_fn=kernel_fn,theta=theta_0)

118.31066424897398

In [75]:
minimize(fun=neg_ll,x0=theta_0,args=(X1,t,kernel_fn))

      fun: 20.792175637627842
 hess_inv: array([[ 6.97177912e+06,  0.00000000e+00,  1.27686011e+07,
         1.88385123e+03, -1.23102409e+02],
       [ 0.00000000e+00,  4.76837158e-07,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.27686011e+07,  0.00000000e+00,  2.33853591e+07,
         3.46226456e+03, -2.25456295e+02],
       [ 1.88385123e+03,  0.00000000e+00,  3.46226456e+03,
         3.38390309e+00, -1.82766046e-02],
       [-1.23102409e+02,  0.00000000e+00, -2.25456295e+02,
        -1.82766046e-02,  2.65617337e-02]])
      jac: array([-3.86238098e-05, -4.76837158e-06,  2.62260437e-05, -1.43861771e-03,
        1.07836723e-02])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 673
      nit: 39
     njev: 96
   status: 2
  success: False
        x: array([1.25163671e+03, 1.58452741e+03, 2.29342511e+03, 5.07289990e+00,
       1.26582348e+00])

In [92]:
import pandas as pd
train_data = pd.read_csv('../data/train_data.csv')
train_data.head()
X = train_data[['x','x_dot','F']]
D_x = X.values.shape[1]
theta_0 = np.random(loc=2,scale = 1,size=D_x+2)
t = train_data[['x_prime','x_dot_prime']]
minimize(fun=neg_ll,x0=theta_0,args=(X.values,t.values,kernel_fn))

TypeError: 'module' object is not callable

## Ref:
1. http://krasserm.github.io/2018/03/19/gaussian-processes/
2. https://papers.nips.cc/paper/2420-gaussian-processes-in-reinforcement-learning.pdf